In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class YOLOv8Backbone(nn.Module):
    def __init__(self):
        super(YOLOv8Backbone, self).__init__()
        # Simplified backbone architecture
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    class Forward:
        def __call__(self, x, model):
            x = F.relu(model.bn1(model.conv1(x)))
            x = model.pool(x)
            x = F.relu(model.bn2(model.conv2(x)))
            x = model.pool(x)
            x = F.relu(model.bn3(model.conv3(x)))
            x = model.pool(x)
            return x

    def forward(self, x):
        forward_instance = self.Forward()
        return forward_instance(x, self)

class YOLOv8Classifier(nn.Module):
    def __init__(self, num_classes):
        super(YOLOv8Classifier, self).__init__()
        self.backbone = YOLOv8Backbone()
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(128, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes)
        )

    class Forward:
        def __call__(self, x, model):
            x = model.backbone(x)
            x = model.classifier(x)
            return x

    def forward(self, x):
        forward_instance = self.Forward()
        return forward_instance(x, self)

# Example usage
num_classes = 2  # Adjust this based on your dataset
model = YOLOv8Classifier(num_classes)

# Print the model architecture
print(model)


YOLOv8Classifier(
  (backbone): YOLOv8Backbone(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): AdaptiveAvgPool2d(output_size=(1, 1))
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=128, out_features=512, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)


In [2]:
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 2

In [3]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

In [4]:
train_dataset = datasets.ImageFolder(root='C:/Users/Aditi Bolakhe/Desktop/drowsy detection/cnn1/training', transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = datasets.ImageFolder(root='C:/Users/Aditi Bolakhe/Desktop/drowsy detection/cnn1/valid', transform=transform)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

In [5]:
model = YOLOv8Classifier(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Print the average loss for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/2], Loss: 0.3789
Epoch [2/2], Loss: 0.1381


In [7]:
# Validate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

Validation Accuracy: 89.75%
